In [1]:
%python -m pip install --upgrade pip


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [ ]:
%pip install numpy
%pip install matplotlib
%pip install opencv-python
%pip install tensorflow==2.11.0


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


<h1>First Run - simple face detection (fake confidence)</h1>

In [11]:
import cv2

# Load a pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

print("Press 'q' to quit the webcam feed.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If frame is read correctly, ret is True
    if not ret:
        print("Failed to grab frame.")
        break

    # Convert frame to grayscale (Haar Cascades work on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect objects (faces in this case)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop over detected faces
    for (x, y, w, h) in faces:
        # Calculate a "confidence score" based on the size of the detected face
        area = w * h  # Area of the bounding box (larger area = higher confidence)
        confidence = min(area / (frame.shape[0] * frame.shape[1]) * 100, 100)  # Relative size in percentage

        # Draw rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Display the "Face" label and confidence score
        label = f"Face: {int(confidence)}%"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Webcam - Object Detection', frame)

    # Check for key press to quit (e.g., 'q' to quit)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Stopping the webcam.")
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Press 'q' to quit the webcam feed.
Stopping the webcam.


<h1>2nd run, extensible, but still fake confidence</h1>

In [13]:
import cv2

# Load default classifiers
# You can extend this list with more classifiers later
classifiers = {
    "face": cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'),
    "eyes": cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml'),
    "smile": cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
}

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

print("Press 'q' to quit the webcam feed.")
print("Adding new object detectors is supported!")

def add_object_detector(name, classifier_path):
    """Add a new object detector dynamically."""
    new_classifier = cv2.CascadeClassifier(classifier_path)
    if new_classifier.empty():
        print(f"Error loading classifier from {classifier_path}")
    else:
        classifiers[name] = new_classifier
        print(f"Added new object detector: {name} from {classifier_path}")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If frame is read correctly, ret is True
    if not ret:
        print("Failed to grab frame.")
        break

    # Convert frame to grayscale (Haar Cascades work on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Loop through all added classifiers and detect objects
    for object_name, classifier in classifiers.items():
        objects = classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Loop over detected objects
        for (x, y, w, h) in objects:
            # Calculate "confidence" based on bounding box area
            area = w * h
            confidence = min(area / (frame.shape[0] * frame.shape[1]) * 100, 100)

            # Draw rectangle around the detected object
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Display the label and confidence score
            label = f"{object_name.capitalize()}: {int(confidence)}%"
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Webcam - Multi Object Detection', frame)

    # Check for key press to quit (e.g., 'q' to quit)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Stopping the webcam.")
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


Press 'q' to quit the webcam feed.
Adding new object detectors is supported!
Stopping the webcam.


<h1>V2: Real confidence using TF, and inputs</h1>

In [ ]:
%conda create --name tfone python=3.10

In [ ]:
%conda activate tfone

In [ ]:
%conda install numpy

In [ ]:
%conda install -c conda-forge tensorflow

In [ ]:
%conda install -c conda-forge opencv

In [2]:
import cv2
import numpy as np
#import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions

# Load the pre-trained MobileNetV2 model
model = MobileNetV2(weights='imagenet')

# Function to get the object classification
def classify_image(frame):
    # Convert frame to image format compatible with the model
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))  # Resize to 224x224
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess for the model
    
    # Get predictions
    predictions = model.predict(img_array)
    
    # Decode predictions and get the most likely object
    decoded_predictions = decode_predictions(predictions, top=1)[0][0]
    
    return decoded_predictions

# Function to run the webcam and get user input for noun
def run_webcam():
    # Open the webcam (default is 0)
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    
    print("Please enter a noun for classification:")
    noun = input("Enter noun: ").lower()

    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
        
        # Classify the current frame from the webcam
        prediction = classify_image(frame)
        predicted_class = prediction[1].lower()  # Predicted object label
        confidence = prediction[2]  # Confidence score
        
        # Check if the predicted class matches the input noun
        match = "yes" if noun in predicted_class else "no"
        
        # Display the classification result
        cv2.putText(frame, f"Predicted: {predicted_class} ({confidence*100:.2f}%)", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.putText(frame, f"Match with '{noun}': {match}", 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show the frame
        cv2.imshow('Webcam Feed', frame)
        
        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the webcam and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    run_webcam()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Brett\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Brett\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Brett\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Brett\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\Brett\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\Brett\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\Brett\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Brett\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Brett\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "c:\Users\Brett\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle